In [ ]:
!pip install transformers sentencepiece torch pandas tqdm konlpy scikit-learn -q
!git clone https://github.com/park1200656/KnuSentiLex.git knusentilexdownload 2>/dev/null || echo "KnuSentiLex already present"

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# ------------------------------------------------------------
# 1️⃣  댓글 데이터 (예: youtube_comments)
# ------------------------------------------------------------
# 반드시 'text' 컬럼이 있어야 합니다.
# 예시)
# youtube_comments = pd.read_csv("youtube_comments.csv")
# columns = ['video_id','text','likeCount','published_at']

# ------------------------------------------------------------
# 2️⃣  감정 분석 함수 (기존 코드에서 가져옴)
# ------------------------------------------------------------

def fused_sent_score(text: str) -> dict:
    """KR-FinBERT + Kiwi + KnuSentiLex 간단 결합 버전"""
    lex = lexicon_sent_score(text)
    fin = finbert_sent_score(text)
    fused = 0.7 * fin["finbert_score"] + 0.3 * lex["lex_score"]

    if fused > 0.2:
        label = "긍정"
    elif fused < -0.2:
        label = "부정"
    else:
        label = "중립"

    return {"text_label": label, "fused_score": float(np.clip(fused, -1, 1))}

# ------------------------------------------------------------
# 3️⃣  댓글별 감정 라벨링 수행
# ------------------------------------------------------------

def add_text_label(df, text_col="text"):
    results = []
    for text in tqdm(df[text_col], desc="댓글 감정 분석중"):
        try:
            result = fused_sent_score(str(text))
        except Exception as e:
            result = {"text_label": "오류", "fused_score": 0.0}
        results.append(result)

    res_df = pd.DataFrame(results)
    df = pd.concat([df.reset_index(drop=True), res_df], axis=1)
    return df

# ------------------------------------------------------------
# 4️⃣  실행
# ------------------------------------------------------------
youtube_comments_labeled = add_text_label(youtube_comments)

# 결과 확인
print(youtube_comments_labeled.head())


댓글 감정 분석중: 100%|██████████| 6/6 [00:00<00:00, 23.28it/s]

  video_id                text text_label  fused_score
0       v1   역시 대단하다 또 유출이네 ㅋㅋ         부정    -0.637439
1       v1  서비스 좋아요. 개선도 빨라졌고요         중립     0.078816
2       v1         아 이건 좀 아닌 듯         중립    -0.001716
3       v2        업데이트 나쁘지 않네요         중립    -0.000021
4       v2          최악이다 진심 실망         부정    -0.697706


In [2]:
# ============================================================
# 1️⃣ 라이브러리 설치
# ============================================================
# (colab 또는 로컬 환경에서 1회만 실행)
# !pip install transformers torch kiwipiepy --upgrade

# ============================================================
# 2️⃣ 라이브러리 임포트
# ============================================================
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from kiwipiepy import Kiwi

# ============================================================
# 3️⃣ 모델 및 토크나이저 불러오기
# ============================================================
MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# ============================================================
# 4️⃣ 감정 예측 함수 정의
# ============================================================
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    label_id = torch.argmax(probs).item()

    # 모델 레이블 정의 (모델 카드에 따라 달라질 수 있음)
    labels = ["부정", "중립", "긍정"] if probs.shape[0] == 3 else ["부정", "긍정"]

    return {
        "text": text,
        "label": labels[label_id],
        "confidence": round(float(probs[label_id]), 3),
        "scores": {labels[i]: round(float(p), 3) for i, p in enumerate(probs)}
    }

# ============================================================
# 5️⃣ 테스트 문장
# ============================================================
samples = [
    "와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.",
    "그렇게 잘 될 줄 알았습니다. 다음번엔 노벨상이라도 타시겠어요.",
    "역시 기대를 저버리지 않으시네요.",
    "작성자에 의해 삭제된 댓글입니다.",
    "당하기만 하는 나라"
]

# ============================================================
# 6️⃣ 실행
# ============================================================
results = [predict_sentiment(text) for text in samples]

for r in results:
    print(f"\n문장: {r['text']}")
    print(f"→ 감정: {r['label']} ({r['confidence']})")
    print(f"세부 점수: {r['scores']}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\speec\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\speec\.cache\huggingface\hub\models--alsgyu--sentiment-analysis-fine-tuned-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]


문장: 와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.
→ 감정: 긍정 (0.915)
세부 점수: {'부정': 0.01, '중립': 0.076, '긍정': 0.915}

문장: 그렇게 잘 될 줄 알았습니다. 다음번엔 노벨상이라도 타시겠어요.
→ 감정: 긍정 (0.953)
세부 점수: {'부정': 0.004, '중립': 0.043, '긍정': 0.953}

문장: 역시 기대를 저버리지 않으시네요.
→ 감정: 중립 (0.481)
세부 점수: {'부정': 0.149, '중립': 0.481, '긍정': 0.37}

문장: 작성자에 의해 삭제된 댓글입니다.
→ 감정: 긍정 (0.514)
세부 점수: {'부정': 0.194, '중립': 0.292, '긍정': 0.514}

문장: 당하기만 하는 나라
→ 감정: 부정 (0.594)
세부 점수: {'부정': 0.594, '중립': 0.294, '긍정': 0.112}


C:\Users\speec\AppData\Local\Temp\ipykernel_25632\2955265296.py:37: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  "confidence": round(float(probs[label_id]), 3),


In [7]:
!git clone https://github.com/park1200656/KnuSentiLex.git knusentilexdownload

Cloning into 'knusentilexdownload'...


In [ ]:
# ============================================================
# KnuSentiLex 사전 + 하이브리드 감정분석 통합 코드
# ============================================================

import os
import pandas as pd
import torch
from kiwipiepy import Kiwi
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from typing import Dict, List, Tuple

# -----------------------------
# 설정
# -----------------------------
KNU_REPO_URL = "https://github.com/park1200656/KnuSentiLex.git"
# ── 위 URL은 예시이며 실제 경로는 저장소 구조 확인 필요
HYBRID_MODEL_CANDIDATES = [
    "snunlp/KR-FinBERT",
    "snunlp/KR-FinBert-Sentiment",
    "beomi/KcELECTRA-base"
]
ALS_MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"

# -----------------------------
# 1) 사전 다운로드 및 로드
# -----------------------------
def download_knu(path: str) -> None:
    os.system(f"wget -O {path} {KNU_REPO_URL}")

def load_knu_lexicon(path: str) -> Dict[str, float]:
    lex = {}
    # 파일이 탭 또는 공백 구분일 가능성
    df = pd.read_csv(path, sep="\t", header=None, names=["word", "polarity"], encoding="utf-8")
    for _, r in df.iterrows():
        word = str(r["word"]).strip()
        try:
            score = float(r["polarity"])
        except:
            continue
        lex[word] = score
    return lex

# 파일 경로 찾기 - 여러 경로 시도
possible_paths = [
    "knusentilexdownload/SentiWord_Dict.txt",
    "knusentilexdownload/KnuSentiLex/SentiWord_Dict.txt",
    "KnuSentiLex_SentiWord_Dict.txt"
]

LEXICON_PATH = None
for path in possible_paths:
    if os.path.exists(path):
        LEXICON_PATH = path
        print(f"✓ 파일 찾음: {path}")
        break

if LEXICON_PATH is None:
    print("⚠ SentiWord_Dict.txt를 찾을 수 없습니다!")
    print(f"가능한 경로들:")
    for p in possible_paths:
        print(f"  - {p}")
    # 기본 경로로 설정
    LEXICON_PATH = possible_paths[0]

LEXICON = load_knu_lexicon(LEXICON_PATH)
print(f"[LOAD] KnuSentiLex entries: {len(LEXICON)}")

# -----------------------------
# 2) 형태소 분석기 초기화
# -----------------------------
kiwi = Kiwi()

# -----------------------------
# 3) 모델 로딩 함수
# -----------------------------
def load_classifier(model_name: str) -> Tuple[AutoTokenizer, AutoModelForSequenceClassification, List[str]]:
    tok = AutoTokenizer.from_pretrained(model_name)
    mdl = AutoModelForSequenceClassification.from_pretrained(model_name)
    with torch.no_grad():
        dummy = tok("테스트 문장", return_tensors="pt", truncation=True, padding=True)
        out = mdl(**dummy)
    n = out.logits.shape[-1]
    if n == 2:
        labels = ["부정", "긍정"]
    elif n == 3:
        labels = ["부정", "중립", "긍정"]
    else:
        labels = [f"class_{i}" for i in range(n)]
    return tok, mdl, labels

def try_load_any(candidates: List[str]) -> Tuple[str, AutoTokenizer, AutoModelForSequenceClassification, List[str]]:
    for name in candidates:
        try:
            tok, mdl, lbls = load_classifier(name)
            print(f"[LOAD] base model loaded: {name} -> {lbls}")
            return name, tok, mdl, lbls
        except Exception as e:
            print(f"[WARN] failed to load {name}: {e}")
            continue
    raise RuntimeError("No base model could be loaded.")

BASE_NAME, base_tok, base_mdl, base_labels = try_load_any(HYBRID_MODEL_CANDIDATES)
als_tok, als_mdl, als_labels = load_classifier(ALS_MODEL_NAME)
print(f"[LOAD] als model loaded: {ALS_MODEL_NAME} -> {als_labels}")

# -----------------------------
# 4) Lexicon 점수 함수
# -----------------------------
def lexicon_score(text: str) -> float:
    tokens = [t.form for t in kiwi.tokenize(text)]
    return sum(LEXICON.get(tok, 0.0) for tok in tokens)

def sarcasm_flag(text: str, lscore: float) -> bool:
    tokens = [t.form for t in kiwi.tokenize(text)]
    positive_markers = {"천재", "대단", "역시", "잘", "노벨상"}
    if any(tok in positive_markers for tok in tokens) and lscore < -0.3:
        return True
    if text.count("...") >= 2 or ("?" in text and "!" in text):
        return True
    return False

# -----------------------------
# 5) 예측 함수
# -----------------------------
@torch.no_grad()
def predict_hf(tok, mdl, labels, text: str) -> Tuple[str, float, Dict[str,float]]:
    inputs = tok(text, return_tensors="pt", truncation=True, padding=True)
    out = mdl(**inputs)
    probs = torch.softmax(out.logits, dim=-1)[0].cpu().numpy().tolist()
    idx = int(torch.argmax(out.logits, dim=-1).item())
    return labels[idx], float(probs[idx]), {labels[i]: round(probs[i],3) for i in range(len(labels))}

def hybrid_predict(text: str, w_model: float = 0.7, w_lex: float = 0.3) -> Dict:
    lscore = lexicon_score(text)
    base_label, base_conf, base_dist = predict_hf(base_tok, base_mdl, base_labels, text)
    signed = (base_dist.get("긍정",0) - base_dist.get("부정",0))
    final_score = w_model * signed + w_lex * lscore
    if "삭제된 댓글" in text or "부적절한 표현" in text:
        label = "기타"
    elif sarcasm_flag(text, lscore):
        label = "반어/비꼼"
    else:
        if final_score > 0.2:
            label = "긍정"
        elif final_score < -0.2:
            label = "분노(강부정)" if lscore < -1.0 else "부정"
        else:
            label = "중립"
    return {
        "label": label,
        "score": round(final_score,3),
        "lex_score": round(lscore,3),
        "base_label": base_label,
        "base_conf": round(base_conf,3),
        "base_dist": base_dist
    }

# -----------------------------
# 6) 비교 실행
# -----------------------------
def compare_models(texts: List[str]) -> None:
    for t in texts:
        hy = hybrid_predict(t)
        als_label, als_conf, als_dist = predict_hf(als_tok, als_mdl, als_labels, t)
        print("\n문장:", t)
        print(">> HYBRID :", hy["label"], f"(score={hy['score']}, lex={hy['lex_score']})", f"base={hy['base_label']}")
        print(">> ALS     :", als_label, f"(conf={als_conf})", "dist=", als_dist)

if __name__ == "__main__":
    sample_comments = [
        "와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.",
        "그렇게 잘 될 줄 알았습니다. 다음번엔 노벨상이라도 타시겠어요.",
        "역시 기대를 저버리지 않으시네요."
    ]
    compare_models(sample_comments)


FileNotFoundError: [Errno 2] No such file or directory: 'KnuSentiLex_SentiWord_Dict.txt'

In [ ]:
# ========== 파일 경로 확인 및 수정 ==========
import os
import glob

print("[파일 경로 확인]")
base_path = "knusentilexdownload"

# 다운로드된 파일 확인
all_files = glob.glob(f"{base_path}/**/*.txt", recursive=True)
print(f"\n찾은 .txt 파일: {len(all_files)}개")
for f in all_files[:10]:
    print(f"  - {f}")

# 주요 파일 확인
key_files = [
    os.path.join(base_path, "pos_pol_word.txt"),
    os.path.join(base_path, "neg_pol_word.txt"),
    os.path.join(base_path, "KnuSentiLex", "pos_pol_word.txt"),
    os.path.join(base_path, "KnuSentiLex", "neg_pol_word.txt"),
]

print("\n[주요 파일 존재 여부]")
for f in key_files:
    exists = "✓" if os.path.exists(f) else "✗"
    print(f"{exists} {f}")
